# Improved Data Engineering Package Validation

This notebook validates packages with robust error handling for complex packages like apache-airflow.

In [ ]:
import sys
import importlib
import subprocess
import pandas as pd
from IPython.display import display, HTML

def check_package_robust(package_name, import_name=None, fallback_imports=None):
    """Robust package checking with multiple fallback strategies"""
    if import_name is None:
        import_name = package_name
    
    # Strategy 1: Try primary import
    try:
        module = importlib.import_module(import_name)
        
        # Try multiple version attributes
        version_attrs = ['__version__', 'VERSION', 'version', '_version']
        version = 'Unknown'
        
        for attr in version_attrs:
            if hasattr(module, attr):
                version_val = getattr(module, attr)
                if version_val:
                    version = str(version_val)
                    break
        
        return {'status': '✅', 'version': version, 'method': 'import'}
        
    except ImportError:
        # Strategy 2: Try fallback imports for complex packages
        if fallback_imports:
            for fallback in fallback_imports:
                try:
                    module = importlib.import_module(fallback)
                    version = getattr(module, '__version__', 'Unknown')
                    return {'status': '✅', 'version': version, 'method': f'fallback: {fallback}'}
                except:
                    continue
        
        # Strategy 3: Check via pip show (package installed but import issues)
        try:
            result = subprocess.run(['pip', 'show', package_name], 
                                  capture_output=True, text=True, timeout=10)
            if result.returncode == 0:
                for line in result.stdout.split('\n'):
                    if line.startswith('Version:'):
                        version = line.split(':', 1)[1].strip()
                        return {'status': '⚠️', 'version': version, 'method': 'pip (import failed)'}
        except Exception:
            pass
            
        return {'status': '❌', 'version': 'Not Found', 'method': 'not installed'}
        
    except Exception as e:
        return {'status': '⚠️', 'version': f'Error: {str(e)[:50]}...', 'method': 'exception'}

print("🔍 Robust Package Validation Starting...\n")

## Package Validation with Fallback Strategies

In [ ]:
# Define packages with their import strategies
packages_config = [
    # Base packages (already in jupyter/pyspark-notebook)
    {'name': 'pyarrow', 'import': 'pyarrow'},
    {'name': 'pyspark', 'import': 'pyspark'},
    
    # Core data processing
    {'name': 'boto3', 'import': 'boto3'},
    
    # Data lake and streaming
    {'name': 'deltalake', 'import': 'deltalake'},
    {'name': 'kafka-python', 'import': 'kafka'},
    {'name': 'minio', 'import': 'minio'},
    {'name': 'lakefs-client', 'import': 'lakefs_client'},
    
    # Data quality and orchestration (complex packages)
    {'name': 'great-expectations', 'import': 'great_expectations'},
    {'name': 'apache-airflow', 'import': 'airflow', 'fallbacks': ['airflow.version', 'airflow.configuration']},
    
    # Model serving and deployment
    {'name': 'bentoml', 'import': 'bentoml'},
    {'name': 'kubeflow-training', 'import': 'kubeflowtraining', 'fallbacks': ['kubeflow.training']},
    {'name': 'kubernetes', 'import': 'kubernetes', 'fallbacks': ['kubernetes.client']},
    
    # Jupyter environment
    {'name': 'ipywidgets', 'import': 'ipywidgets'},
]

print(f"Checking {len(packages_config)} packages with robust validation...\n")

results = []
for pkg_config in packages_config:
    pkg_name = pkg_config['name']
    import_name = pkg_config['import']
    fallbacks = pkg_config.get('fallbacks', None)
    
    print(f"Checking {pkg_name}...", end=" ")
    result = check_package_robust(pkg_name, import_name, fallbacks)
    
    results.append({
        'Package': pkg_name,
        'Status': result['status'],
        'Version': result['version'],
        'Method': result['method']
    })
    
    print(f"{result['status']} ({result['method']})")

print("\n" + "="*60)
df = pd.DataFrame(results)
display(HTML(df.to_html(index=False, escape=False)))

## Detailed Analysis

In [ ]:
# Analyze results by status
status_counts = df['Status'].value_counts()
method_counts = df['Method'].value_counts()

print("📊 Installation Analysis:")
print(f"✅ Working: {status_counts.get('✅', 0)}")
print(f"⚠️ Issues: {status_counts.get('⚠️', 0)}")
print(f"❌ Missing: {status_counts.get('❌', 0)}")

total = len(results)
success_rate = (status_counts.get('✅', 0) / total) * 100
print(f"\n📈 Success Rate: {success_rate:.1f}%")

# Show problematic packages
problematic = df[df['Status'] != '✅']
if not problematic.empty:
    print("\n⚠️ Packages needing attention:")
    for _, row in problematic.iterrows():
        print(f"  • {row['Package']}: {row['Version']} ({row['Method']})")
else:
    print("\n🎉 All packages are working correctly!")

print("\n🔧 Detection Methods Used:")
for method, count in method_counts.items():
    print(f"  • {method}: {count} packages")

## Functionality Tests for Complex Packages

In [ ]:
print("🧪 Testing complex package functionality...\n")

functionality_tests = []

# Test Apache Airflow (complex package)
print("Testing Apache Airflow...")
try:
    # Try multiple import strategies
    import airflow
    from airflow import __version__ as airflow_version
    functionality_tests.append(('Apache Airflow', '✅', f'Core import successful (v{airflow_version})'))
    
    # Test configuration access
    try:
        from airflow.configuration import conf
        functionality_tests.append(('Airflow Config', '✅', 'Configuration accessible'))
    except Exception as e:
        functionality_tests.append(('Airflow Config', '⚠️', f'Config issue: {str(e)[:40]}...'))
        
except Exception as e:
    functionality_tests.append(('Apache Airflow', '❌', f'Import failed: {str(e)[:40]}...'))

# Test Great Expectations
print("Testing Great Expectations...")
try:
    import great_expectations as gx
    # Try to create a basic context
    functionality_tests.append(('Great Expectations', '✅', f'Import successful (v{gx.__version__})'))
except Exception as e:
    functionality_tests.append(('Great Expectations', '❌', f'Failed: {str(e)[:40]}...'))

# Test Kubeflow Training
print("Testing Kubeflow Training...")
try:
    import kubeflowtraining
    functionality_tests.append(('Kubeflow Training', '✅', f'Import successful'))
    
    # Test training module
    try:
        from kubeflow import training
        functionality_tests.append(('Training Module', '✅', 'Training module accessible'))
    except Exception as e:
        functionality_tests.append(('Training Module', '⚠️', f'Module issue: {str(e)[:40]}...'))
        
except Exception as e:
    functionality_tests.append(('Kubeflow Training', '❌', f'Failed: {str(e)[:40]}...'))

# Test BentoML
print("Testing BentoML...")
try:
    import bentoml
    functionality_tests.append(('BentoML', '✅', f'Import successful (v{bentoml.__version__})'))
except Exception as e:
    functionality_tests.append(('BentoML', '❌', f'Failed: {str(e)[:40]}...'))

# Display results
print("\n" + "="*50)
func_df = pd.DataFrame(functionality_tests, columns=['Component', 'Status', 'Result'])
display(HTML(func_df.to_html(index=False, escape=False)))

print("\n✨ Functionality testing complete!")

## Environment Summary

In [ ]:
import platform
import os

print("🖥️ Environment Information:")
print(f"Python: {sys.version.split()[0]}")
print(f"Platform: {platform.system()} {platform.release()}")
print(f"Architecture: {platform.machine()}")

# Check for Coder environment
if os.getenv('CODER_AGENT_TOKEN'):
    print(f"Environment: Coder Workspace")
    print(f"Workspace: Data Engineering Template")
else:
    print(f"Environment: Standard Jupyter")

print(f"\n📦 Package Summary:")
print(f"Total packages checked: {len(results)}")
print(f"Validation strategies used: {len(method_counts)}")
print(f"Complex packages handled: apache-airflow, kubeflow-training, kubernetes")

print(f"\n🎯 This robust validation handles complex packages with multiple import strategies.")